# 2. Build a Recommendation System for Purchase Data

The scope of this notebook is 

- Code the Scoring Function
- Unit Test the Score
- Build the Dash Applimcation



## Settings

In [25]:
#Model Tracking
import mlflow
from mlflow.tracking import MlflowClient

from minio import Minio
from minio.error import ResponseError

from surprise import dump

import json
import pickle
import os

#Utils
from pprint import pprint

In [21]:
#Enviroment variables

outmodels = '../models/model.pkl'

## Download the model from the registry

In [22]:
# client = MlflowClient()
# for regmodel in client.list_registered_models():
#     regmodel_info = dict(regmodel)

# pprint(regmodel_info, indent=3)

# # run_id = regmodel_info['latest_versions'][0].run_id
# # path = "/".join(regmodel_info['latest_versions'][0].source.split('/')[2:])

# run_id = 'f4d1b856bb0a4abf9c8e42e6f7eb688f'
# path = '/artifacts/1/019488b1b2f44793a30de8b3eeae1b72/artifacts/model'

# client.download_artifacts(run_id, path, dst_path=outmodels)

In [23]:
minioClient = Minio(os.environ['MLFLOW_S3_ENDPOINT_URL'].split('//')[1],
                  access_key=os.environ['AWS_ACCESS_KEY_ID'],
                  secret_key=os.environ['AWS_SECRET_ACCESS_KEY'],
                  secure=False)

# Check buckets 
buckets = minioClient.list_buckets()
for bucket in buckets:
    print(bucket.name, bucket.creation_date)
    
#Check for content
objects = minioClient.list_objects_v2(
        bucket_name = bucket.name, 
        recursive=True)

for obj in objects:
    print(obj.bucket_name, obj.object_name.encode('utf-8'), obj.last_modified,
          obj.etag, obj.size, obj.content_type)
    
#Download Model
try:
    minioClient.fget_object(bucket_name=obj.bucket_name, object_name=b'artifacts/1/019488b1b2f44793a30de8b3eeae1b72/artifacts/model/modeloyx7rlbx.pkl', file_path=outmodels)
except ResponseError as err:
    print(err)

mlflow 2020-05-23 13:36:30.006000+00:00
mlflow b'artifacts/1/019488b1b2f44793a30de8b3eeae1b72/artifacts/best-predicitionsi_xr5k95.csv' 2020-05-23 13:38:37.032000+00:00 d73c1423020b49ef6a278fee61418fcf 565 None
mlflow b'artifacts/1/019488b1b2f44793a30de8b3eeae1b72/artifacts/model/modeloyx7rlbx.pkl' 2020-05-23 13:38:36.905000+00:00 c7970f420fa35db450890811b04f531e 4980195 None
mlflow b'artifacts/1/099eb04c85d749ed8e94479a66dfa015/artifacts/best-predicitionsciwc88ba.csv' 2020-05-23 13:38:34.813000+00:00 d73c1423020b49ef6a278fee61418fcf 565 None
mlflow b'artifacts/1/099eb04c85d749ed8e94479a66dfa015/artifacts/model/modelnlvif4t6.pkl' 2020-05-23 13:38:34.685000+00:00 dc2c528a570ea9460dba7465a412ffde 4980195 None
mlflow b'artifacts/1/210019fabbc848f1976097d203aee3f8/artifacts/best-predicitions9_cl4akk.csv' 2020-05-23 13:38:38.116000+00:00 d73c1423020b49ef6a278fee61418fcf 565 None
mlflow b'artifacts/1/210019fabbc848f1976097d203aee3f8/artifacts/model/modelnqs0y7ff.pkl' 2020-05-23 13:38:37.99200

### Load and predict

In [26]:
predictions, algo = dump.load(outmodels)

In [28]:
algo

### Single value

In [ ]:
test_sample = data_prep_4.head()
test_sample.head()

In [ ]:
uid = str(1007)  # raw user id (as in the ratings file). They are **strings**!
iid = str(0)  # raw item id (as in the ratings file). They are **strings**!

# get a prediction for specific users and items.
pred = algo_best.predict(uid, iid, r_ui=13, verbose=True)

### Score the entire dataset

In [ ]:
data_prep_5['predictions'] = data_prep_5.apply(lambda row:algo_best.predict(row['userID'], 
                     row['itemID'], row['rating']), axis = 1)
data_prep_5.head()
# predictions=list(data_prep_5['predictions'])

### Top 10 Recommended Items


In [ ]:
def get_top_n(predictions, n=10):
    
    '''Return the top-N recommendation for each user from a set of predictions.
    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.
    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

#     # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

def get_top_n_ui(top, ui):
    try:
        return {k:v for k,v in top.items() if ui==k}
    except ValueError: # user was not part of the trainset
        return 0

In [ ]:
print(get_top_n_ui(get_top_n(predictions, n=10), '20400'))